In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
import numpy as np
import pickle

import tensorflow as tf
from tensorflow.keras import layers, models

from build_model_ed import build_model_1, build_model_2,build_model_3

In [4]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [5]:
#load data
test_size = 0.1
stride_step = 200
norm_factor=1
dtype = 'uint8'

filename = f'save_data//e_d_train_all_str_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
#filename = f'save_data//e_d_train_Puck_str_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
#load train data
with open(filename, 'rb') as f:
    train_list,frames_data_train = pickle.load(f)
    
print(f'Training dataset shape: {frames_data_train.shape}')
print(f'Training dataset dtype: {frames_data_train.dtype}')

Training dataset shape: (19250, 2048, 160, 3)
Training dataset dtype: uint8


In [ ]:
#training parameters
epochs = 2000
#for ps 2 and du 100
batch_size = 3
#for ps 4 and du 10
batch_size = 8 
#for ps 4 and du 5
batch_size = 16
#for model_ed_3
batch_size = 8
#for everything else
batch_size = 16
validation_split = 0.2

dense_units_list = [100,60,40,20,10]
dense_units_list = [10,20,40,60,100]
dense_units_list=[5,10,20,40]
dense_units_list=[10]


for dense_units in dense_units_list:
    ifBatchNorm = True
    kernel_size = (3,3)
    if 0:
        nn_blocks_list = [
            [
                [[16,16], kernel_size, ifBatchNorm, (2,1)],
                [[32,32], kernel_size, ifBatchNorm, (2,1)],
                [[64, 64], kernel_size, ifBatchNorm, (2,1)],
                [[128,128], kernel_size, ifBatchNorm, (2,1)], 
            ],  
            [
                [[16,16], kernel_size, ifBatchNorm, (4,1)],
                [[32,32], kernel_size, ifBatchNorm, (4,1)],
                [[64, 64], kernel_size, ifBatchNorm, (4,1)],
                [[128,128], kernel_size, ifBatchNorm, (4,1)],            
            ]                 
        ]
    else:
        if 0:
            nn_blocks_list = [
                [
                    [[16,16], kernel_size, ifBatchNorm, (4,1)],
                    [[32,32], kernel_size, ifBatchNorm, (4,1)],
                    [[64, 64], kernel_size, ifBatchNorm, (4,1)],
                    [[128,128], kernel_size, ifBatchNorm, (4,1)],            
                ],
            ]
        else:
            if 1:
                nn_blocks_list = [
                    [
                        [[4], kernel_size, ifBatchNorm, (2,1)],
                        [[8], kernel_size, ifBatchNorm, (2,1)],
                        [[16], kernel_size, ifBatchNorm, (2,1)],
                        [[32], kernel_size, ifBatchNorm, (2,1)],  
                        [[64], kernel_size, ifBatchNorm, (2,1)], 
                        [[100], kernel_size, ifBatchNorm, (2,2)], 
                        [[120], kernel_size, ifBatchNorm, (2,2)], 
                        [[140], kernel_size, ifBatchNorm, (2,2)], 
                    ],
                ]
            else:
                nn_blocks_list = [
                    [
                        [[20], kernel_size, ifBatchNorm, (2,1)],
                        [[40], kernel_size, ifBatchNorm, (2,1)],
                        [[60], kernel_size, ifBatchNorm, (2,1)],
                        [[80], kernel_size, ifBatchNorm, (2,1)],  
                        [[100], kernel_size, ifBatchNorm, (2,1)], 
                        [[120], kernel_size, ifBatchNorm, (2,1)], 
                        [[140], kernel_size, ifBatchNorm, (2,1)], 
                        [[160], kernel_size, ifBatchNorm, (2,2)], 
                    ],
                ]
                
                 
        
    
    for nn_blocks in nn_blocks_list:
        
        #poolsize
        ps = nn_blocks[0][-1][0]
        
        if (ps==4) and (dense_units==10 or dense_units==5):
            continue
        
        model_name = f'model_ed_3_ps_{ps}_bn_{ifBatchNorm}_du_{dense_units}'

        if 1:
            #save data
            save_data = [dense_units,ifBatchNorm,kernel_size,nn_blocks]
            file_name = f'saved_models//stride_{stride_step}//{model_name}_settings.pkl'
            with open(file_name, 'wb') as f:
                pickle.dump(save_data, f)


        input_shape = (2048, stride_step, 3)

        loss = 'mean_squared_error'
        optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.1, name="Adadelta")
        #optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.01, name="SGD")


        model = build_model_3(input_shape, dense_units, nn_blocks)

        model.compile(loss=loss, 
                      optimizer=optimizer, )
        model.summary() 
        
        if 1:
            #define callbacks
            # Write TensorBoard logs
            log_dir = f'./tensorboard/stride_{stride_step}/logs/{model_name}'
            tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
            #Stop training when no improvement
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                              patience=10, 
                                                              restore_best_weights=True)
            #Reduce learning rate when a metric has stopped improving
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5)
            callbacks = [tensorboard, early_stopping, reduce_lr]
        
            model.fit(
                x=frames_data_train,
                y=frames_data_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_split=validation_split,
                callbacks=callbacks,
            )



            model.save_weights(f'saved_models/stride_{stride_step}/{model_name}/{model_name}')
            
        del model
        
        

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048, 160, 3)]    0         
                                                                 
 rescaling (Rescaling)       (None, 2048, 160, 3)      0         
                                                                 
 conv2d (Conv2D)             (None, 2048, 160, 4)      112       
                                                                 
 batch_normalization (BatchN  (None, 2048, 160, 4)     16        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1024, 160, 4)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1024, 160, 8)      296   

 up_sampling2d_5 (UpSampling  (None, 512, 160, 16)     0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 512, 160, 8)       1160      
                                                                 
 batch_normalization_14 (Bat  (None, 512, 160, 8)      32        
 chNormalization)                                                
                                                                 
 up_sampling2d_6 (UpSampling  (None, 1024, 160, 8)     0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 1024, 160, 4)      292       
                                                                 
 batch_normalization_15 (Bat  (None, 1024, 160, 4)     16        
 chNormalization)                                                
          

In [ ]:
---------------------------------------------------------------------------
InternalError                             Traceback (most recent call last)
Input In [6], in <cell line: 19>()
    107     reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5)
    108     callbacks = [tensorboard, early_stopping, reduce_lr]
--> 110     model.fit(
    111         x=frames_data_train,
    112         y=frames_data_train,
    113         batch_size=batch_size,
    114         epochs=epochs,
    115         validation_split=validation_split,
    116         callbacks=callbacks,
    117     )
    121     model.save_weights(f'saved_models/{model_name}/{model_name}')
    123 del model

File ~/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py:67, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     65 except Exception as e:  # pylint: disable=broad-except
     66   filtered_tb = _process_traceback_frames(e.__traceback__)
---> 67   raise e.with_traceback(filtered_tb) from None
     68 finally:
     69   del filtered_tb

File ~/.local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py:102, in convert_to_eager_tensor(value, ctx, dtype)
    100     dtype = dtypes.as_dtype(dtype).as_datatype_enum
    101 ctx.ensure_initialized()
--> 102 return ops.EagerTensor(value, ctx.device_name, dtype)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
#tensorboard results

#model_name = f'model_encoder_decoder_test_3'

log_dir = f'./tensorboard/logs/{model_name}'
%tensorboard --logdir $log_dir